In [1]:
!pip install yfinance ta scikit-learn joblib

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=5eca1f3ab77f56e4d41d4e5ebdd5129a1d866475fa56d6126afe4b449953b819
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [5]:
import yfinance as yf
import ta
import pandas as pd

df = yf.download("^NSEI", period="6mo", interval="1h")

# 🔥 FIX MULTI-INDEX COLUMNS HERE
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

# Add indicators
df["rsi"] = ta.momentum.RSIIndicator(df["Close"]).rsi()
df["ema20"] = ta.trend.EMAIndicator(df["Close"], window=20).ema_indicator()

macd = ta.trend.MACD(df["Close"])
df["macd"] = macd.macd()
df["signal"] = macd.macd_signal()

df = df.dropna()
df.tail()

/tmp/ipython-input-1947062649.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("^NSEI", period="6mo", interval="1h")
[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume,rsi,ema20,macd,signal
Datetime,,,,,,,,,
2025-12-09 04:45:00+00:00,25864.400391,25868.150391,25778.849609,25784.050781,0,37.886991,26005.613382,-48.818488,-19.561260
2025-12-09 05:45:00+00:00,25893.050781,25899.699219,25817.800781,25865.000000,0,40.504113,25994.893134,-51.450694,-25.939147
2025-12-09 06:45:00+00:00,25876.199219,25923.150391,25852.000000,25892.150391,0,39.451192,25983.588952,-54.270914,-31.605501
2025-12-09 07:45:00+00:00,25885.500000,25907.400391,25864.349609,25875.550781,0,40.372505,25974.247147,-55.120074,-36.308415
2025-12-09 08:45:00+00:00,25824.300781,25886.099609,25821.599609,25884.650391,0,36.443090,25959.966540,-60.039211,-41.054574


In [6]:
df["target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)

In [7]:
# Create target (next candle up/down)
df["target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
df = df.dropna()

# Features
features = ["rsi", "ema20", "macd", "signal"]
X = df[features]
y = df["target"]

In [9]:
from sklearn.model_selection import train_test_split
# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Train model
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    random_state=42
)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, n_estimators=200, random_state=42)

In [11]:
# Test accuracy
preds = model.predict(X_test)
accuracy = accuracy_score(y_test, preds)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.47928994082840237


In [12]:
# Save model
joblib.dump(model, "nifty_rf.joblib")
print("Model saved as nifty_rf.joblib")

Model saved as nifty_rf.joblib
